In [2]:
# -*- coding: utf-8 -*-

# ライブラリインポート
import pandas as pd

In [3]:
"""ソフトバンクデータの読み込み・前処理を行う関数"""
def process_softbank_data(file_path):
    # データの読み込み
    softbank = pd.read_csv(file_path, encoding="utf-8")

    # 年月日を適切な形式に変換
    softbank["FormattedDate"] = pd.to_datetime(
        softbank["Year"].astype(str) + "/" +
        softbank["Date"].str.extract(r"(\d+)月(\d+)日")[0].fillna('0') +
        "/" + softbank["Date"].str.extract(r"(\d+)月(\d+)日")[1].fillna('0'),
        format="%Y/%m/%d"
    )

    # 曜日を追加
    softbank["Weekday"] = softbank["FormattedDate"].dt.day_name()

    return softbank

In [4]:
def process_weather_data(file_path):
    """天気データの読み込み・前処理を行う関数"""
    # データの読み込み
    weather = pd.read_csv(file_path, encoding="shift_jis")

    # 必要な列のみを抽出
    weather = weather.iloc[:, [0, 1, 5, 8]]

    # 列名をリネーム
    weather.columns = [
        "yyyy/mm/dd",
        "Total_Precipitation (mm)",
        "Average_Temperature (℃)",
        "Average_CloudCover (%)"
    ]

    # Date列をdatetime型に変換
    weather["yyyy/mm/dd"] = pd.to_datetime(weather["yyyy/mm/dd"])

    return weather

In [5]:
def merge_datasets(softbank_df, weather_df):
    """ソフトバンクデータと天気データをマージする関数"""
    # マージ処理: 結合キーをdatetime型に合わせる
    merged_df = pd.merge(softbank_df, weather_df, how="left", left_on="FormattedDate", right_on="yyyy/mm/dd")
    return merged_df

In [6]:
def save_to_csv(df, output_path):
    """データフレームをCSVとして出力する関数"""
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [7]:
# ファイルパスの設定
softbank_file = "/content/softbank_audience_full_data.csv"
weather_file = "/content/weather.csv"
output_file = "/content/final_data.csv"

# 前処理の実行
softbank_df = process_softbank_data(softbank_file)
weather_df = process_weather_data(weather_file)

# データのマージ
final_df = merge_datasets(softbank_df, weather_df)

# CSV出力
save_to_csv(final_df, output_file)

FileNotFoundError: [Errno 2] No such file or directory: '/content/softbank_audience_full_data.csv'

In [ ]:
final_df